## <font color=#6B49F5> A Simple Implementation of FedAvg with PyTorch on IID Data </font> 
Please see https://towardsdatascience.com/federated-learning-a-simple-implementation-of-fedavg-federated-averaging-with-pytorch-90187c9c9577 for more details.

In [66]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE
import copy
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = "{:,.4f}".format
sm = SMOTE(random_state=42)

In [67]:
THREAT_TYPE = 'threat_type'
THREAT_HL = 'threat_hl'

learning_rate = 0.01
numEpoch = 20
batch_size = 32
momentum = 0.9
print_amount=5
number_of_slices = 5
isSmote = False

file_name = "kdd-slice-specific-centralized.txt"
file = open(file_name, "w")

data_path = "D:\\learning\\PyTorch\\NSL_KDD-master\\"

colnames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
            'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
            'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
            'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
            'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
            'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate', 'threat_type']

In [68]:
df_train = pd.read_csv(data_path + "KDDTrain+.csv", header = None)
df_train = df_train.iloc[:, :-1]

df_test = pd.read_csv(data_path + "KDDTest+.csv", header = None)
df_test = df_test.iloc[:, :-1]

df_train.columns = colnames
df_test.columns = colnames

df_train.loc[(df_train['threat_type'] == 'back'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'buffer_overflow'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'ftp_write'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'guess_passwd'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'imap'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'ipsweep'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'land'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'loadmodule'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'multihop'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'neptune'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'nmap'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'perl'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'phf'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'pod'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'portsweep'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'rootkit'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'satan'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'smurf'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'spy'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'teardrop'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'warezclient'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'warezmaster'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'normal'), 'threat_type'] = 0
df_train.loc[(df_train['threat_type'] == 'unknown'), 'threat_type'] = 6

df_test.loc[(df_test['threat_type'] == 'back'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'buffer_overflow'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'ftp_write'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'guess_passwd'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'imap'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'ipsweep'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'land'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'loadmodule'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'multihop'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'neptune'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'nmap'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'perl'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'phf'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'pod'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'portsweep'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'rootkit'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'satan'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'smurf'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'spy'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'teardrop'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'warezclient'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'warezmaster'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'normal'), 'threat_type'] = 0
df_test.loc[(df_test['threat_type'] == 'unknown'), 'threat_type'] = 6
df_test.loc[(df_test['threat_type'] == 'mscan'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'apache2'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'snmpgetattack'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'processtable'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'httptunnel'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'ps'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'snmpguess'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'mailbomb'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'named'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'sendmail'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'xterm'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'xlock'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'xsnoop'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'sqlattack'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'udpstorm'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'saint'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'worm'), 'threat_type'] = 1

df_full = pd.concat([df_train, df_test])

print('Attack types in full set: \n', df_full[THREAT_TYPE].value_counts())

Attack types in full set: 
 0    77053
1    53387
4    14077
3     3880
2      119
Name: threat_type, dtype: int64


In [69]:

print('Before normalization shape of data set : ', df_full.shape)
threat_type_df = df_full['threat_type'].copy()
# Considering numerical columns
# 34 numerical columns are considered for training
numerical_colmanes = ['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot',
                      'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
                      'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'count',
                      'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
                      'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
                      'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                      'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                      'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

numerical_df_full = df_full[numerical_colmanes].copy()
print(numerical_df_full.shape)
#
# # Lets remove the numerical columns with constant value
numerical_df_full = numerical_df_full.loc[:, (numerical_df_full != numerical_df_full.iloc[0]).any()]
#
# # lets scale the values for each column from [0,1]
# # N.B. we dont have any negative values]
final_df_full = numerical_df_full / numerical_df_full.max()
print(final_df_full.shape)

df_normalized = pd.concat([final_df_full, threat_type_df], axis=1)
print('After normalization shape of data set: ', df_normalized.shape)
print(df_normalized[THREAT_TYPE].value_counts())

Before normalization shape of data set :  (148516, 42)
(148516, 34)
(148516, 33)
After normalization shape of data set:  (148516, 34)
0    77053
1    53387
4    14077
3     3880
2      119
Name: threat_type, dtype: int64


In [70]:
df_normal = df_normalized[df_normalized['threat_type'] == 0]
df_dos = df_normalized[df_normalized['threat_type'] == 1]
df_u2r = df_normalized[df_normalized['threat_type'] == 2]
df_r2u = df_normalized[df_normalized['threat_type'] == 3]
df_probe = df_normalized[df_normalized['threat_type'] == 4]

print(df_normal.shape, df_dos.shape, df_u2r.shape, df_r2u.shape, df_probe.shape)

(77053, 34) (53387, 34) (119, 34) (3880, 34) (14077, 34)


In [71]:
def divide_train_test(df, propotion=0.1):
    
    df_train = []
    df_test = []
    for key,val in df[THREAT_TYPE].value_counts().iteritems():
        df_part = df[df['threat_type'] == key]
        df_test.append(df_part[0: int(df_part.shape[0]*propotion)])
        df_train.append(df_part[int(df_part.shape[0]*propotion):df_part.shape[0]])
        
    return df_train,df_test
    

In [72]:
def get_data_for_slices(df_train, number_of_slices, isSmote=False, x_name="x_train", y_name="y_train"):
    df_normal_s1 = df_normal[int(df_normal.shape[0]*0/number_of_slices):int(df_normal.shape[0]*(1)/number_of_slices)]
    df_normal_s2 = df_normal[int(df_normal.shape[0]*1/number_of_slices):int(df_normal.shape[0]*(2)/number_of_slices)]
    df_normal_s3 = df_normal[int(df_normal.shape[0]*2/number_of_slices):int(df_normal.shape[0]*(3)/number_of_slices)]
    df_normal_s4 = df_normal[int(df_normal.shape[0]*3/number_of_slices):int(df_normal.shape[0]*(4)/number_of_slices)]
    df_normal_s5 = df_normal[int(df_normal.shape[0]*4/number_of_slices):int(df_normal.shape[0]*(5)/number_of_slices)]

    df_s1 = pd.concat([df_normal_s1,df_dos])
    df_s2 = pd.concat([df_normal_s2,df_u2r])
    df_s3 = pd.concat([df_normal_s3,df_r2u])
    df_s4 = pd.concat([df_normal_s4,df_probe])
    df_s5 = df_normal_s5

    x_data_dict= dict()
    y_data_dict= dict()

    y_s1 = (df_s1.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s1, return_counts=True)
    print('slice 1: ')
    print(dict(zip(unique, counts)))
    y_info1 = torch.tensor(y_s1).type(torch.LongTensor)
    x_info1 = torch.tensor(df_s1.values).float()

    y_s2 = (df_s2.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s2, return_counts=True)
    print('slice 2: ')
    print(dict(zip(unique, counts)))
    y_info2 = torch.tensor(y_s2).type(torch.LongTensor)
    x_info2 = torch.tensor(df_s2.values).float()

    y_s3 = (df_s3.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s3, return_counts=True)
    print('slice 3: ')
    print( dict(zip(unique, counts)))
    y_info3 = torch.tensor(y_s3).type(torch.LongTensor)
    x_info3 = torch.tensor(df_s3.values).float()

    y_s4 = (df_s4.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s4, return_counts=True)
    print('slice 4: ')
    print(dict(zip(unique, counts)))
    y_info4 = torch.tensor(y_s4).type(torch.LongTensor)
    x_info4 = torch.tensor(df_s4.values).float()

    y_s5 = (df_s5.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s5, return_counts=True)
    print('slice 5: ')
    print(dict(zip(unique, counts)))
    y_info5 = torch.tensor(y_s5).type(torch.LongTensor)
    x_info5 = torch.tensor(df_s5.values).float()


    x_data_dict.update({"x_train0" : x_info1})
    y_data_dict.update({"y_train0" : y_info1})

    x_data_dict.update({"x_train1" : x_info2})
    y_data_dict.update({"y_train1" : y_info2})

    x_data_dict.update({"x_train2" : x_info3})
    y_data_dict.update({"y_train2" : y_info3})

    x_data_dict.update({"x_train3" : x_info4})
    y_data_dict.update({"y_train3" : y_info4})

    x_data_dict.update({"x_train4" : x_info5})
    y_data_dict.update({"y_train4" : y_info5})
    
    return x_data_dict, y_data_dict 
        
        

In [73]:
df_train, df_test = divide_train_test(df_normalized,propotion=0.1)
# print('Value counts in train set : ')
# df_train[THREAT_TYPE].value_counts()
# print('Value counts in test set : ')
# print(df_test[THREAT_TYPE].value_counts())

x_train_dict, y_train_dict = get_data_for_slices(df_train, number_of_slices, isSmote)

for y_train_s in y_train_dict:
    print(y_train_s)
df_test = pd.concat(df_test)
y_test = df_test.pop(THREAT_TYPE).values
x_test = df_test.values

print('Test set size is : x => ' + str(x_test.shape) + ' y => ' + str(y_test.shape))
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test.astype('int')).type(torch.LongTensor)

inputs = x_test.shape[1]
outputs = 5

slice 1: 
{0: 15410, 1: 53387}
slice 2: 
{0: 15411, 2: 119}
slice 3: 
{0: 15410, 3: 3880}
slice 4: 
{0: 15411, 4: 14077}
slice 5: 
{0: 15411}
y_train0
y_train1
y_train2
y_train3
y_train4
Test set size is : x => (14849, 33) y => (14849,)


--------------------------
### <span style="background-color:#F087F9"> Classification Model </span> 

In [74]:
class Net2nn(nn.Module):
    def __init__(self, inputs, outputs):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(inputs,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,outputs)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [75]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [76]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [77]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [78]:
def confusion_mat(model, test_loader, desc = 'none'):
    
    print(desc)
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in test_loader:
        output = model(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
#     df_cm = pd.DataFrame(cf_matrix, index=[i for i in Counter(y_test)],
#                          columns=[i for i in Counter(y_test)])
#     plt.figure(1)
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.figure(figsize=(12, 7))

#     sn.heatmap(df_cm, annot=True).set(xlabel='Predicted label', ylabel='True label')
#     plt.savefig('D:\\learning\\PyTorch\\experiment\\cf\\cf_fl_'+str(self.number_of_slices)+'.png')
    print('confusion matrix for normal scenario for slices : ' + str(number_of_slices))
    print(cf_matrix)
    file.write('\ncf matrix for slice :' + str(number_of_slices))
    file.write('\n'+str(cf_matrix))

---------------------------------
### <span style="background-color:#F087F9"> Functions for Federated Averaging </span> 

In [79]:
def create_model_optimizer_criterion_dict(number_of_slices):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_slices):
        model_name="model"+str(i)
        model_info=Net2nn(inputs, outputs)
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

In [80]:
def get_averaged_weights(model_dict, number_of_slices):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_slices):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_slices
        fc1_mean_bias = fc1_mean_bias/ number_of_slices
    
        fc2_mean_weight =fc2_mean_weight/number_of_slices
        fc2_mean_bias = fc2_mean_bias/ number_of_slices
    
        fc3_mean_weight =fc3_mean_weight/number_of_slices
        fc3_mean_bias = fc3_mean_bias/ number_of_slices
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [81]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_slices=number_of_slices)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [82]:
def compare_local_and_merged_model_performance(number_of_slices):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_slices,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_slices):
    
        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [83]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices):
    with torch.no_grad():
        for i in range(number_of_slices):
            print('Updating model :' + name_of_models[i] )
            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone() 
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
    
    return model_dict

In [84]:
def start_train_end_node_process(number_of_slices):
    for i in range (number_of_slices): 

        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

#         valid_ds = TensorDataset(x_valid_dict[name_of_x_valid_sets[i]], y_valid_dict[name_of_y_valid_sets[i]])
#         valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)
        
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
#             valid_loss, valid_accuracy = validation(model, valid_dl, criterion)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))

In [85]:

def start_train_end_node_process_without_print(number_of_slices):
    for i in range (number_of_slices): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
        confusion_mat(model, test_dl, desc = 'Confusion matrix for ' + name_of_models[i])

In [86]:
def start_train_end_node_process_print_some(number_of_slices, print_amount):
    for i in range (number_of_slices): 
        
        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], 
                                 y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        if i<print_amount:
            print("Subset" ,i)
            
        for epoch in range(numEpoch):
        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
            if i<print_amount:        
                print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy)) 
                
        confusion_mat(model, test_dl, desc = 'Confusion matrix for ' + name_of_models[i])

In [87]:
# x_train, y_train, x_valid, y_valid,x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))


----------------

### <span style="background-color:#F087F9"> Let's examine what would the performance of the centralized model be if the data were not distributed to nodes at all? </span>   

The model used in this example is very simple, different things can be done to improve model performance, such as using more complex models, increasing epoch or hyperparameter tuning. However, the purpose here is to compare the performance of the main model that is formed by combining the parameters of the local models trained on their own data with a centralized model that trained on all training data. In this way, we can gain insight into the capacity of federated learning.


In [88]:
# initial_model = Net2nn()
# initial_optimizer = torch.optim.SGD(initial_model.parameters(), lr=0.01, momentum=0.9)
# initial_criterion = nn.CrossEntropyLoss()

centralized_model = Net2nn(inputs, outputs)
centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
centralized_criterion = nn.CrossEntropyLoss()

In [89]:
print("------ Centralized Model - 4 ------")
file.write('Centralized model 4')

train_acc4 = []
test_acc4 = []
train_loss4 = []
test_loss4 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model4 = Net2nn(inputs,outputs)
centralized_optimizer4 = torch.optim.SGD(centralized_model4.parameters(), lr=0.01, momentum=0.9)
centralized_criterion4 = nn.CrossEntropyLoss()
#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(4)
y_name = 'y_train' + str(4)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model4, train_dl, centralized_criterion4, centralized_optimizer4)
    central_test_loss, central_test_accuracy = validation(centralized_model4, test_dl, centralized_criterion4)

    train_acc4.append(central_train_accuracy)
    train_loss4.append(central_train_loss)
    test_acc4.append(central_test_accuracy)
    test_loss4.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model4, test_dl)

print(centralized_model4.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc4)/len(train_acc4)))
print('Mean test accuracy: ' + str(sum(test_acc4)/len(test_acc4)))


------ Centralized Model - 4 ------
 | train accuracy:  1.0000 | test accuracy:  0.5189
none
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [5338    0    0    0    0]
 [  11    0    0    0    0]
 [ 388    0    0    0    0]
 [1407    0    0    0    0]]
tensor([[-0.0495, -0.0449,  0.0294,  0.0419, -0.0119]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9996885341639089
Mean test accuracy: 0.5188901609535995


In [90]:
file.write('\nCentralized model 4 Mean train accuracy: ' + str(sum(train_acc4)/len(train_acc4)))
file.write('\nCentralized model 4 Mean test accuracy: ' + str(sum(test_acc4)/len(test_acc4)))

59

In [91]:
print("------ Centralized Model - 0 ------")
file.write('Centralized model 0')


train_acc0 = []
test_acc0 = []
train_loss0 = []
test_loss0 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model0 = Net2nn(inputs, outputs)
centralized_optimizer0 = torch.optim.SGD(centralized_model0.parameters(), lr=0.01, momentum=0.9)
centralized_criterion0 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(0)
y_name = 'y_train' + str(0)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model0, train_dl, centralized_criterion0, centralized_optimizer0)
    central_test_loss, central_test_accuracy = validation(centralized_model0, test_dl, centralized_criterion0)

    train_acc0.append(central_train_accuracy)
    train_loss0.append(central_train_loss)
    test_acc0.append(central_test_accuracy)
    test_loss0.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model0, test_dl)

print(centralized_model0.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc0)/len(train_acc0)))
print('Mean test accuracy: ' + str(sum(test_acc0)/len(test_acc0)))


------ Centralized Model - 0 ------
 | train accuracy:  0.9933 | test accuracy:  0.8618
none
confusion matrix for normal scenario for slices : 5
[[7463  242    0    0    0]
 [   4 5334    0    0    0]
 [  11    0    0    0    0]
 [ 356   32    0    0    0]
 [ 791  616    0    0    0]]
tensor([[-0.0503, -0.0749,  0.0441, -0.0568,  0.0374]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9879595040481414
Mean test accuracy: 0.8631961748265876


In [92]:
file.write('\nCentralized model 0 Mean train accuracy: ' + str(sum(train_acc0)/len(train_acc0)))
file.write('\nCentralized model 0 Mean test accuracy: ' + str(sum(test_acc0)/len(test_acc0)))

59

In [93]:
print("------ Centralized Model - 1 ------")
file.write('Centralized model 1')


train_acc1 = []
test_acc1 = []
train_loss1 = []
test_loss1 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model1 = Net2nn(inputs, outputs)
centralized_optimizer1 = torch.optim.SGD(centralized_model1.parameters(), lr=0.01, momentum=0.9)
centralized_criterion1 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(1)
y_name = 'y_train' + str(1)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model1, train_dl, centralized_criterion1, centralized_optimizer1)
    central_test_loss, central_test_accuracy = validation(centralized_model1, test_dl, centralized_criterion1)

    train_acc1.append(central_train_accuracy)
    train_loss1.append(central_train_loss)
    test_acc1.append(central_test_accuracy)
    test_loss1.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model1, test_dl)

print(centralized_model1.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc1)/len(train_acc1)))
print('Mean test accuracy: ' + str(sum(test_acc1)/len(test_acc1)))


------ Centralized Model - 1 ------
 | train accuracy:  0.9956 | test accuracy:  0.5190
none
confusion matrix for normal scenario for slices : 5
[[7701    0    4    0    0]
 [5338    0    0    0    0]
 [   5    0    6    0    0]
 [ 387    0    1    0    0]
 [1407    0    0    0    0]]
tensor([[-0.0109,  0.0201,  0.0362,  0.0352, -0.0091]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9942916934964583
Mean test accuracy: 0.5190147484679105


In [94]:
file.write('\nCentralized model 1 Mean train accuracy: ' + str(sum(train_acc1)/len(train_acc1)))
file.write('\nCentralized model 1 Mean test accuracy: ' + str(sum(test_acc1)/len(test_acc1)))

59

In [95]:
print("------ Centralized Model - 2 ------")
file.write('Centralized model 2')

train_acc2 = []
test_acc2 = []
train_loss2 = []
test_loss2 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model2 = Net2nn(inputs, outputs)
centralized_optimizer2 = torch.optim.SGD(centralized_model2.parameters(), lr=0.01, momentum=0.9)
centralized_criterion2 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(2)
y_name = 'y_train' + str(2)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model2, train_dl, centralized_criterion2, centralized_optimizer2)
    central_test_loss, central_test_accuracy = validation(centralized_model2, test_dl, centralized_criterion2)

    train_acc2.append(central_train_accuracy)
    train_loss2.append(central_train_loss)
    test_acc2.append(central_test_accuracy)
    test_loss2.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model2, test_dl)

print(centralized_model2.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc2)/len(train_acc2)))
print('Mean test accuracy: ' + str(sum(test_acc2)/len(test_acc2)))


------ Centralized Model - 2 ------
 | train accuracy:  0.9187 | test accuracy:  0.5249
none
confusion matrix for normal scenario for slices : 5
[[7428    0    0  277    0]
 [5321    0    0   17    0]
 [   2    0    0    9    0]
 [  22    0    0  366    0]
 [1163    0    0  244    0]]
tensor([[ 0.0099,  0.0783,  0.0455,  0.0199, -0.0422]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9052177293934681
Mean test accuracy: 0.5182537544615801


In [96]:
file.write('\nCentralized model 2 Mean train accuracy: ' + str(sum(train_acc2)/len(train_acc2)))
file.write('\nCentralized model 2 Mean test accuracy: ' + str(sum(test_acc2)/len(test_acc2)))

59

In [97]:
print("------ Centralized Model - 3 ------")
file.write('Centralized model 3')

train_acc3 = []
test_acc3 = []
train_loss3 = []
test_loss3 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model3 = Net2nn(inputs, outputs)
centralized_optimizer3 = torch.optim.SGD(centralized_model3.parameters(), lr=0.01, momentum=0.9)
centralized_criterion3 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(3)
y_name = 'y_train' + str(3)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model3, train_dl, centralized_criterion3, centralized_optimizer3)
    central_test_loss, central_test_accuracy = validation(centralized_model3, test_dl, centralized_criterion3)

    train_acc3.append(central_train_accuracy)
    train_loss3.append(central_train_loss)
    test_acc3.append(central_test_accuracy)
    test_loss3.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model3, test_dl)

print(centralized_model3.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc3)/len(train_acc3)))
print('Mean test accuracy: ' + str(sum(test_acc3)/len(test_acc3)))


------ Centralized Model - 3 ------
 | train accuracy:  0.9850 | test accuracy:  0.6072
none
confusion matrix for normal scenario for slices : 5
[[7653    0    0    0   52]
 [ 495    0    0    0 4843]
 [  11    0    0    0    0]
 [ 246    0    0    0  142]
 [  43    0    0    0 1364]]
tensor([[ 0.0115,  0.1051,  0.0557, -0.0163,  0.0203]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9780436109603908
Mean test accuracy: 0.6015017846319617


In [98]:
file.write('\nCentralized model 3 Mean train accuracy: ' + str(sum(train_acc3)/len(train_acc3)))
file.write('\nCentralized model 3 Mean test accuracy: ' + str(sum(test_acc3)/len(test_acc3)))

59


----------------
-----------------
**Data is distributed to nodes**

<!-- ### <span style="background-color:#F087F9"> Datanın nodelara dağıtılması </span>    -->

In [34]:
print(x_train_dict["x_train1"].shape, y_train_dict["y_train1"].shape)
print(x_test.shape, y_test.shape)

torch.Size([15530, 33]) torch.Size([15530])
torch.Size([14849, 33]) torch.Size([14849])


**Main model is created**

In [35]:
main_model = Net2nn(inputs,outputs)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

**Models,optimizers and loss functions in nodes are defined**

In [36]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_slices)

**Keys of dicts are being made iterable**

In [37]:
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())

name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

print(name_of_x_train_sets)
print(name_of_y_train_sets)
print("\n ------------")
print(name_of_models)
print(name_of_optimizers)
print(name_of_criterions)

['x_train0', 'x_train1', 'x_train2', 'x_train3', 'x_train4']
['y_train0', 'y_train1', 'y_train2', 'y_train3', 'y_train4']

 ------------
['model0', 'model1', 'model2', 'model3', 'model4']
['optimizer0', 'optimizer1', 'optimizer2', 'optimizer3', 'optimizer4']
['criterion0', 'criterion1', 'criterion2', 'criterion3', 'criterion4']


In [38]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0097, -0.0600,  0.0491,  0.0106, -0.0333]],
       grad_fn=<SliceBackward>)


**Parameters of main model are sent to nodes**  
Since the parameters of the main model and parameters of all local models in the nodes are randomly initialized, all these parameters will be different from each other. For this reason, the main model sends its parameters to the nodes before the training of local models in the nodes begins. You can check the weights below.

In [39]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4


In [40]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])
print(model_dict["model0"].fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])
print(model_dict["model2"].fc2.weight[0,0:5])
print(model_dict["model3"].fc2.weight[0,0:5])
print(model_dict["model4"].fc2.weight[0,0:5])

tensor([[ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268]],
       grad_fn=<SliceBackward>)
tensor([ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268], grad_fn=<SliceBackward>)
tensor([ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268], grad_fn=<SliceBackward>)
tensor([ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268], grad_fn=<SliceBackward>)
tensor([ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268], grad_fn=<SliceBackward>)
tensor([ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268], grad_fn=<SliceBackward>)


**Models in the nodes are trained**

In [41]:
# start_train_end_node_process()
start_train_end_node_process_print_some(number_of_slices, print_amount)

Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 0.95705 | test accuracy: 0.85642
epoch:   2 | train accuracy: 0.97903 | test accuracy: 0.86558
epoch:   3 | train accuracy: 0.98317 | test accuracy: 0.86039
epoch:   4 | train accuracy: 0.98567 | test accuracy: 0.86774
epoch:   5 | train accuracy: 0.98695 | test accuracy: 0.86774
epoch:   6 | train accuracy: 0.98879 | test accuracy: 0.86740
epoch:   7 | train accuracy: 0.98930 | test accuracy: 0.86396
epoch:   8 | train accuracy: 0.98951 | test accuracy: 0.86565
epoch:   9 | train accuracy: 0.99084 | test accuracy: 0.87056
epoch:  10 | train accuracy: 0.99051 | test accuracy: 0.86807
epoch:  11 | train accuracy: 0.99046 | test accuracy: 0.86935
epoch:  12 | train accuracy: 0.99145 | test accuracy: 0.87043
epoch:  13 | train accuracy: 0.99203 | test accuracy: 0.86935
epoch:  14 | train accuracy: 0.99228 | test accuracy: 0.86915
epoch:  15 | train accuracy: 0.99257 | test accuracy: 0.87231
epoch:  16 | train accuracy: 0

In [42]:
## As you can see, wieghts of local models are updated after training process
print(main_model.fc2.weight[0,0:5])
print(model_dict["model0"].fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])
print(model_dict["model2"].fc2.weight[0,0:5])
print(model_dict["model3"].fc2.weight[0,0:5])
print(model_dict["model4"].fc2.weight[0,0:5])

tensor([ 0.0166,  0.0359, -0.0435, -0.0656,  0.0268], grad_fn=<SliceBackward>)
tensor([ 0.0031,  0.0602, -0.0781, -0.0823, -0.0016], grad_fn=<SliceBackward>)
tensor([-0.0003,  0.0424, -0.0547, -0.0714,  0.0454], grad_fn=<SliceBackward>)
tensor([ 0.0314,  0.0224, -0.0295, -0.0691,  0.0272], grad_fn=<SliceBackward>)
tensor([-0.0491,  0.0243, -0.0249, -0.1317,  0.1323], grad_fn=<SliceBackward>)
tensor([ 0.0244,  0.0478, -0.0432, -0.0629,  0.0297], grad_fn=<SliceBackward>)


### Let's compare the performance of federated main model, individual local models and centralized model  

**Federated main model vs individual local models before 1st iteration (on distributed test set)**  
Since main model is randomly initialized and no action taken on it yet, its performance is very poor. Please before_acc_table

In [43]:
before_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
before_test_loss, before_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nbefore training main model')
confusion_mat(main_model, test_dl)

main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
print(main_model.fc2.weight[0,0:5])
after_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
after_test_loss, after_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nafter training main model')
confusion_mat(main_model, test_dl)

none
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [5338    0    0    0    0]
 [  11    0    0    0    0]
 [ 388    0    0    0    0]
 [1407    0    0    0    0]]
tensor([ 0.0019,  0.0394, -0.0461, -0.0835,  0.0466], grad_fn=<SliceBackward>)
none
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [5338    0    0    0    0]
 [  11    0    0    0    0]
 [ 388    0    0    0    0]
 [1407    0    0    0    0]]


In [44]:
print("Federated main model vs individual local models before FedAvg first iteration")
file.write('\nBefore training federated')
file.write('\n'+str(before_acc_table))
before_acc_table

Federated main model vs individual local models before FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.8718,0.5189
1,sample 1,0.5191,0.5189
2,sample 2,0.5163,0.5189
3,sample 3,0.6015,0.5189
4,sample 4,0.5189,0.5189


In [45]:
print("Federated main model vs individual local models after FedAvg first iteration")
file.write('\nAfter training federated')
file.write('\n'+str(after_acc_table))
after_acc_table

Federated main model vs individual local models after FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.8718,0.5189
1,sample 1,0.5191,0.5189
2,sample 2,0.5163,0.5189
3,sample 3,0.6015,0.5189
4,sample 4,0.5189,0.5189


**Federated main model vs centralized model before 1st iteration (on all test data)**  
Please be aware that the centralized model gets approximately %98 accuracy on all test data.

In [46]:
print("Before 1st iteration main model accuracy on all test data: {:7.4f}".format(before_test_accuracy))
print("After 1st iteration main model accuracy on all test data: {:7.4f}".format(after_test_accuracy))
print("Centralized model accuracy on all test data: {:7.4f}".format(central_test_accuracy))

Before 1st iteration main model accuracy on all test data:  0.5189
After 1st iteration main model accuracy on all test data:  0.5189
Centralized model accuracy on all test data:  0.5191


This is a single iteration, we can send the weights of the main model back to the nodes and repeat the above steps.
Now let's check how the performance of the main model improves when we repeat the iteration 10 more times.

In [47]:
for i in range(100):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)
    start_train_end_node_process_print_some(number_of_slices, print_amount)
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))   

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 0.97357 | test accuracy: 0.86659
epoch:   2 | train accuracy: 0.98503 | test accuracy: 0.86901
epoch:   3 | train accuracy: 0.98795 | test accuracy: 0.86518
epoch:   4 | train accuracy: 0.98907 | test accuracy: 0.87050
epoch:   5 | train accuracy: 0.99078 | test accuracy: 0.87198
epoch:   6 | train accuracy: 0.99076 | test accuracy: 0.86827
epoch:   7 | train accuracy: 0.99092 | test accuracy: 0.87130
epoch:   8 | train accuracy: 0.99176 | test accuracy: 0.87191
epoch:   9 | train accuracy: 0.99186 | test accuracy: 0.86800
epoch:  10 | train accuracy: 0.99199 | test accuracy: 0.87050
epoch:  11 | train accuracy: 0.99222 | test accuracy: 0.86652
epoch:  12 | train accuracy: 0.99310 | test accuracy: 0.87124
epoch:  13 | train accuracy: 0.99317 | test accuracy: 0.86922
epoch:  14 | train accuracy: 0.99279 | 

epoch:   7 | train accuracy: 0.99247 | test accuracy: 0.79864
epoch:   8 | train accuracy: 0.99201 | test accuracy: 0.87077
epoch:   9 | train accuracy: 0.99215 | test accuracy: 0.87252
epoch:  10 | train accuracy: 0.99273 | test accuracy: 0.86538
epoch:  11 | train accuracy: 0.99328 | test accuracy: 0.87218
epoch:  12 | train accuracy: 0.99294 | test accuracy: 0.86800
epoch:  13 | train accuracy: 0.99363 | test accuracy: 0.87036
epoch:  14 | train accuracy: 0.99343 | test accuracy: 0.87171
epoch:  15 | train accuracy: 0.99347 | test accuracy: 0.86976
epoch:  16 | train accuracy: 0.99456 | test accuracy: 0.87279
epoch:  17 | train accuracy: 0.99404 | test accuracy: 0.87265
epoch:  18 | train accuracy: 0.99395 | test accuracy: 0.87312
epoch:  19 | train accuracy: 0.99424 | test accuracy: 0.87056
epoch:  20 | train accuracy: 0.99471 | test accuracy: 0.87393
Confusion matrix for model0
confusion matrix for normal scenario for slices : 5
[[7644   61    0    0    0]
 [   5 5333    0    0   

epoch:  16 | train accuracy: 0.99455 | test accuracy: 0.87332
epoch:  17 | train accuracy: 0.99459 | test accuracy: 0.87339
epoch:  18 | train accuracy: 0.99491 | test accuracy: 0.87231
epoch:  19 | train accuracy: 0.99411 | test accuracy: 0.86605
epoch:  20 | train accuracy: 0.99462 | test accuracy: 0.87346
Confusion matrix for model0
confusion matrix for normal scenario for slices : 5
[[7649   56    0    0    0]
 [  17 5321    0    0    0]
 [  11    0    0    0    0]
 [ 361   27    0    0    0]
 [ 791  616    0    0    0]]
Federated learning for slice 2
Subset 1
epoch:   1 | train accuracy: 0.99330 | test accuracy: 0.85359
epoch:   2 | train accuracy: 0.99524 | test accuracy: 0.83426
epoch:   3 | train accuracy: 0.99543 | test accuracy: 0.69412
epoch:   4 | train accuracy: 0.99543 | test accuracy: 0.52643
epoch:   5 | train accuracy: 0.99569 | test accuracy: 0.51889
epoch:   6 | train accuracy: 0.99569 | test accuracy: 0.51916
epoch:   7 | train accuracy: 0.99575 | test accuracy: 0.5

epoch:   1 | train accuracy: 0.99363 | test accuracy: 0.86363
epoch:   2 | train accuracy: 0.99549 | test accuracy: 0.84639
epoch:   3 | train accuracy: 0.99524 | test accuracy: 0.81406
epoch:   4 | train accuracy: 0.99562 | test accuracy: 0.77931
epoch:   5 | train accuracy: 0.99556 | test accuracy: 0.73035
epoch:   6 | train accuracy: 0.99581 | test accuracy: 0.66658
epoch:   7 | train accuracy: 0.99569 | test accuracy: 0.56980
epoch:   8 | train accuracy: 0.99581 | test accuracy: 0.52764
epoch:   9 | train accuracy: 0.99594 | test accuracy: 0.52717
epoch:  10 | train accuracy: 0.99614 | test accuracy: 0.52461
epoch:  11 | train accuracy: 0.99601 | test accuracy: 0.52125
epoch:  12 | train accuracy: 0.99594 | test accuracy: 0.51882
epoch:  13 | train accuracy: 0.99594 | test accuracy: 0.51909
epoch:  14 | train accuracy: 0.99607 | test accuracy: 0.51916
epoch:  15 | train accuracy: 0.99614 | test accuracy: 0.51902
epoch:  16 | train accuracy: 0.99614 | test accuracy: 0.51916
epoch:  

epoch:  10 | train accuracy: 0.99601 | test accuracy: 0.74813
epoch:  11 | train accuracy: 0.99588 | test accuracy: 0.71830
epoch:  12 | train accuracy: 0.99620 | test accuracy: 0.71877
epoch:  13 | train accuracy: 0.99620 | test accuracy: 0.66173
epoch:  14 | train accuracy: 0.99614 | test accuracy: 0.64220
epoch:  15 | train accuracy: 0.99614 | test accuracy: 0.52993
epoch:  16 | train accuracy: 0.99614 | test accuracy: 0.52960
epoch:  17 | train accuracy: 0.99588 | test accuracy: 0.52872
epoch:  18 | train accuracy: 0.99620 | test accuracy: 0.52812
epoch:  19 | train accuracy: 0.99627 | test accuracy: 0.52744
epoch:  20 | train accuracy: 0.99620 | test accuracy: 0.52657
Confusion matrix for model1
confusion matrix for normal scenario for slices : 5
[[7699    0    6    0    0]
 [ 529    0 4809    0    0]
 [   5    0    6    0    0]
 [ 387    0    1    0    0]
 [1112   30  151    0  114]]
Federated learning for slice 3
Subset 2
epoch:   1 | train accuracy: 0.91223 | test accuracy: 0.7

...

epoch:  13 | train accuracy: 0.96615 | test accuracy: 0.91009
epoch:  14 | train accuracy: 0.96708 | test accuracy: 0.90760
epoch:  15 | train accuracy: 0.96553 | test accuracy: 0.89440
epoch:  16 | train accuracy: 0.96407 | test accuracy: 0.89898
epoch:  17 | train accuracy: 0.96464 | test accuracy: 0.92013
epoch:  18 | train accuracy: 0.96667 | test accuracy: 0.90350
epoch:  19 | train accuracy: 0.96501 | test accuracy: 0.90464
epoch:  20 | train accuracy: 0.96594 | test accuracy: 0.90733
Confusion matrix for model2
confusion matrix for normal scenario for slices : 5
[[7616    0    0   89    0]
 [ 488 4788    0   56    6]
 [   2    0    0    9    0]
 [  13    0    0  375    0]
 [ 217    7    0  489  694]]
Federated learning for slice 4
Subset 3
epoch:   1 | train accuracy: 0.99356 | test accuracy: 0.61210
epoch:   2 | train accuracy: 0.99471 | test accuracy: 0.61270
epoch:   3 | train accuracy: 0.99535 | test accuracy: 0.61243
epoch:   4 | train accuracy: 0.99549 | test accuracy: 0.6

confusion matrix for normal scenario for slices : 5
[[7605    0    0  100    0]
 [ 478 4825    0   26    9]
 [   3    0    0    8    0]
 [  14    0    0  374    0]
 [ 332   26    0  321  728]]
Federated learning for slice 4
Subset 3
epoch:   1 | train accuracy: 0.99261 | test accuracy: 0.61284
epoch:   2 | train accuracy: 0.99464 | test accuracy: 0.61358
epoch:   3 | train accuracy: 0.99491 | test accuracy: 0.61277
epoch:   4 | train accuracy: 0.99556 | test accuracy: 0.61001
epoch:   5 | train accuracy: 0.99539 | test accuracy: 0.61203
epoch:   6 | train accuracy: 0.99556 | test accuracy: 0.61041
epoch:   7 | train accuracy: 0.99563 | test accuracy: 0.61122
epoch:   8 | train accuracy: 0.99539 | test accuracy: 0.61176
epoch:   9 | train accuracy: 0.99576 | test accuracy: 0.61156
epoch:  10 | train accuracy: 0.99542 | test accuracy: 0.61243
epoch:  11 | train accuracy: 0.99579 | test accuracy: 0.60994
epoch:  12 | train accuracy: 0.99573 | test accuracy: 0.61183
epoch:  13 | train accu

epoch:   6 | train accuracy: 0.99518 | test accuracy: 0.61230
epoch:   7 | train accuracy: 0.99508 | test accuracy: 0.61135
epoch:   8 | train accuracy: 0.99542 | test accuracy: 0.61095
epoch:   9 | train accuracy: 0.99569 | test accuracy: 0.61129
epoch:  10 | train accuracy: 0.99576 | test accuracy: 0.61156
epoch:  11 | train accuracy: 0.99559 | test accuracy: 0.61169
epoch:  12 | train accuracy: 0.99607 | test accuracy: 0.61142
epoch:  13 | train accuracy: 0.99539 | test accuracy: 0.61102
epoch:  14 | train accuracy: 0.99579 | test accuracy: 0.61156
epoch:  15 | train accuracy: 0.99559 | test accuracy: 0.61210
epoch:  16 | train accuracy: 0.99556 | test accuracy: 0.61236
epoch:  17 | train accuracy: 0.99563 | test accuracy: 0.61250
epoch:  18 | train accuracy: 0.99590 | test accuracy: 0.61216
epoch:  19 | train accuracy: 0.99559 | test accuracy: 0.61189
epoch:  20 | train accuracy: 0.99596 | test accuracy: 0.61284
Confusion matrix for model3
confusion matrix for normal scenario for s

epoch:  15 | train accuracy: 0.99576 | test accuracy: 0.61230
epoch:  16 | train accuracy: 0.99576 | test accuracy: 0.61082
epoch:  17 | train accuracy: 0.99610 | test accuracy: 0.61290
epoch:  18 | train accuracy: 0.99586 | test accuracy: 0.61169
epoch:  19 | train accuracy: 0.99505 | test accuracy: 0.61236
epoch:  20 | train accuracy: 0.99552 | test accuracy: 0.61203
Confusion matrix for model3
confusion matrix for normal scenario for slices : 5
[[7669    0    0    0   36]
 [ 262   31    0    0 5045]
 [  10    0    0    0    1]
 [ 345    1    0    0   42]
 [  19    0    0    0 1388]]
Federated learning for slice 5
Subset 4
epoch:   1 | train accuracy: 0.99974 | test accuracy: 0.74214
epoch:   2 | train accuracy: 1.00000 | test accuracy: 0.73244
epoch:   3 | train accuracy: 1.00000 | test accuracy: 0.72490
epoch:   4 | train accuracy: 1.00000 | test accuracy: 0.72025
epoch:   5 | train accuracy: 1.00000 | test accuracy: 0.71567
epoch:   6 | train accuracy: 1.00000 | test accuracy: 0.7

epoch:   1 | train accuracy: 0.99974 | test accuracy: 0.77790
epoch:   2 | train accuracy: 1.00000 | test accuracy: 0.77049
epoch:   3 | train accuracy: 1.00000 | test accuracy: 0.76503
epoch:   4 | train accuracy: 1.00000 | test accuracy: 0.76093
epoch:   5 | train accuracy: 1.00000 | test accuracy: 0.75635
epoch:   6 | train accuracy: 1.00000 | test accuracy: 0.75285
epoch:   7 | train accuracy: 1.00000 | test accuracy: 0.74988
epoch:   8 | train accuracy: 1.00000 | test accuracy: 0.74699
epoch:   9 | train accuracy: 1.00000 | test accuracy: 0.74476
epoch:  10 | train accuracy: 1.00000 | test accuracy: 0.74234
epoch:  11 | train accuracy: 1.00000 | test accuracy: 0.74045
epoch:  12 | train accuracy: 1.00000 | test accuracy: 0.73837
epoch:  13 | train accuracy: 1.00000 | test accuracy: 0.73641
epoch:  14 | train accuracy: 1.00000 | test accuracy: 0.73466
epoch:  15 | train accuracy: 1.00000 | test accuracy: 0.73257
epoch:  16 | train accuracy: 1.00000 | test accuracy: 0.73136
epoch:  

In [48]:
file = open(file_name, "a")
confusion_mat(main_model, test_dl, desc = 'Confusion matrix for after 10 rounds of the dataset')

Confusion matrix for after 10 rounds of the dataset
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [ 276 5049    0    0   13]
 [   6    0    5    0    0]
 [ 377    2    1    8    0]
 [ 217    9    0    1 1180]]


The accuracy of the centralized model was calculated as approximately 98%. The accuracy of the main model obtained by FedAvg method started from 85% and improved to 94%. In this case, we can say that although the main model obtained by FedAvg method was trained without seeing the data, its performance cannot be underestimated.

In [49]:
file.close()